In [12]:
import numpy as np
import scipy as sp
from scipy.special import expit
from sklearn.metrics import normalized_mutual_info_score
import random


# -------------------------- Algorithm 1: Dynamic Multi-Layer Hypergraph Construction -------------------------
class DynamicHypergraph:
    def __init__(self, nodes, layers, initial_hyperedges, time_windows):
        self.nodes = nodes  # Set of entities {v1, ..., vn}
        self.layers = layers  # Set of functional layers {l1, ..., lm}
        self.current_hyperedges = initial_hyperedges  # Initial hyperedge set
        self.time_windows = time_windows  # Time window sequence
        # Intra-layer adjacency matrices (initialized by layer)
        self.adj_matrices = {layer: np.zeros((len(nodes), len(nodes))) for layer in layers}
        # Coupling tensor T (layer × layer × node)
        self.coupling_tensor = np.zeros((len(layers), len(layers), len(nodes)))
        # Index mappings (for matrix operations)
        self.layer_to_idx = {layer: i for i, layer in enumerate(layers)}
        self.node_to_idx = {node: i for i, node in enumerate(nodes)}

    def update_hyperedge_weights(self, current_time, lambda_short=0.4, lambda_long=0.15, prune_threshold=0.1):
        """Update hyperedge weights (exponential decay) and prune low-weight hyperedges"""
        updated_hyperedges = {}
        for he_id, (he_attrs, layer, create_time, weight) in self.current_hyperedges.items():
            time_diff = current_time - create_time
            # Select decay rate based on hyperedge type
            if he_attrs.get('type') == 'transient':
                new_weight = weight * np.exp(-lambda_short * time_diff)
            else:
                new_weight = weight * np.exp(-lambda_long * time_diff)
            # Prune hyperedges with weight below threshold
            if new_weight >= prune_threshold:
                updated_hyperedges[he_id] = (he_attrs, layer, create_time, new_weight)
        self.current_hyperedges = updated_hyperedges

    def detect_new_events(self, current_time, event_rate=0.1):
        """Simulate new event detection (generate new hyperedges)"""
        new_hyperedges = {}
        new_event_count = max(1, int(len(self.current_hyperedges) * event_rate))
        for i in range(new_event_count):
            he_id = f"he_new_{current_time}_{i}"
            layer = random.choice(list(self.layers))
            selected_nodes = random.sample(self.nodes, k=random.randint(2, 5))
            he_type = 'transient' if random.random() < 0.3 else 'strategic'
            new_hyperedges[he_id] = (
                {'nodes': selected_nodes, 'type': he_type}, 
                layer, 
                current_time, 
                1.0  # Initial weight
            )
        self.current_hyperedges.update(new_hyperedges)

    def update_adjacency_matrices(self):
        """Update intra-layer adjacency matrices"""
        for layer in self.layers:
            adj_matrix = np.zeros((len(self.nodes), len(self.nodes)))
            for (he_attrs, he_layer, create_time, weight) in self.current_hyperedges.values():
                if he_layer != layer:
                    continue
                node_indices = [self.node_to_idx[node] for node in he_attrs['nodes']]
                for i in node_indices:
                    for j in node_indices:
                        if i != j:
                            adj_matrix[i, j] += weight
            self.adj_matrices[layer] = adj_matrix

    def update_coupling_tensor(self, community_structure, eta=0.1):
        """Update coupling tensor (inter-layer community correlation)"""
        num_layers = len(self.layers)
        for t_idx in range(num_layers):
            for s_idx in range(num_layers):
                if t_idx == s_idx:
                    continue
                layer_t = list(self.layers)[t_idx]
                layer_s = list(self.layers)[s_idx]
                comm_t = community_structure[layer_t]
                comm_s = community_structure[layer_s]
                intersect_nodes = set(comm_t) & set(comm_s)
                for node in intersect_nodes:
                    node_idx = self.node_to_idx[node]
                    self.coupling_tensor[t_idx, s_idx, node_idx] += eta * len(intersect_nodes) / len(self.nodes)

    def construct_hypergraph(self, current_time):
        """Complete hypergraph construction workflow"""
        self.update_hyperedge_weights(current_time)
        self.detect_new_events(current_time)
        self.update_adjacency_matrices()
        return self.current_hyperedges, self.adj_matrices, self.coupling_tensor


# -------------------------- Algorithm 2: Multi-Objective Function Evaluation -------------------------
def compute_hypergraph_modularity(hyperedges, community_structure, gamma=1.0, omega=1.0):
    """Compute hypergraph modularity (Qh)"""
    total_weight = sum(weight for (he_attrs, layer, create_time, weight) in hyperedges.values())
    if total_weight == 0:
        return 0.0
    
    modularity = 0.0
    for (he_attrs, layer, create_time, weight) in hyperedges.values():
        he_nodes = he_attrs['nodes']
        comm_count = {comm_id: 0 for comm_id in community_structure.keys()}
        for node in he_nodes:
            for comm_id, comm_nodes in community_structure.items():
                if node in comm_nodes:
                    comm_count[comm_id] += 1
                    break
        dominant_comm = max(comm_count.keys(), key=lambda x: comm_count[x])
        overlap_ratio = comm_count[dominant_comm] / len(he_nodes)
        modularity += weight * (overlap_ratio - gamma * (len(he_nodes) / len(community_structure)) ** omega)
    
    return modularity / total_weight


def compute_interlayer_consistency(community_structure):
    """Compute inter-layer consistency (Jc)"""
    layers = list(community_structure.keys())
    num_layers = len(layers)
    if num_layers < 2:
        return 1.0
    
    jaccard_sum = 0.0
    for i in range(num_layers):
        for j in range(i + 1, num_layers):
            comm_i = community_structure[layers[i]]
            comm_j = community_structure[layers[j]]
            intersect = len(set(comm_i) & set(comm_j))
            union = len(set(comm_i) | set(comm_j))
            jaccard_sum += intersect / union if union > 0 else 0.0
    avg_jaccard = jaccard_sum / (num_layers * (num_layers - 1) / 2)
    
    all_nodes = []
    all_labels = []
    for layer_idx, (layer, comm_nodes) in enumerate(community_structure.items()):
        all_nodes.extend(comm_nodes)
        all_labels.extend([layer_idx] * len(comm_nodes))
    nmi = normalized_mutual_info_score(all_labels, all_labels)
    
    return (avg_jaccard + nmi) / 2


def compute_dynamic_stability(prev_community, curr_community, eta=0.2):
    """Compute dynamic stability (Sd)"""
    if not prev_community:
        return 1.0
    
    prev_node_labels = {}
    for comm_id, comm_nodes in prev_community.items():
        for node in comm_nodes:
            prev_node_labels[node] = comm_id
    
    curr_node_labels = {}
    for comm_id, comm_nodes in curr_community.items():
        for node in comm_nodes:
            curr_node_labels[node] = comm_id
    
    common_nodes = set(prev_node_labels.keys()) & set(curr_node_labels.keys())
    if not common_nodes:
        return 0.0
    
    prev_labels = [prev_node_labels[node] for node in common_nodes]
    curr_labels = [curr_node_labels[node] for node in common_nodes]
    nmi = normalized_mutual_info_score(prev_labels, curr_labels)
    
    prev_comm_ids = set(prev_community.keys())
    curr_comm_ids = set(curr_community.keys())
    new_comm_count = len(curr_comm_ids - prev_comm_ids)
    reorganization_penalty = eta * (new_comm_count / len(curr_comm_ids)) if len(curr_comm_ids) > 0 else 0.0
    
    return max(nmi - reorganization_penalty, 0.0)


def compute_resource_efficiency(hyperedges, role_hierarchy):
    """Compute resource efficiency (Re)"""
    efficiency = 0.0
    alpha = 0.5
    beta = 0.5
    
    for (he_attrs, layer, create_time, weight) in hyperedges.values():
        he_nodes = he_attrs['nodes']
        if len(he_nodes) < 2:
            continue
        
        layer_span = 1
        node_roles = [role_hierarchy.get(node, 1) for node in he_nodes]
        role_diff = max(node_roles) - min(node_roles)
        efficiency += weight * (alpha * layer_span + beta * role_diff)
    
    return efficiency


def evaluate_multi_objectives(hypergraph, community_structure, prev_community, role_hierarchy):
    """Multi-objective evaluation"""
    hyperedges, _, _ = hypergraph
    qh = compute_hypergraph_modularity(hyperedges, community_structure)
    jc = compute_interlayer_consistency(community_structure)
    sd = compute_dynamic_stability(prev_community, community_structure)
    re = compute_resource_efficiency(hyperedges, role_hierarchy)
    return np.array([qh, jc, sd, re])


# -------------------------- Algorithm 3: NSGA-III Community Detection -------------------------
class NSGAIIICommunityDetector:
    def __init__(self, hypergraph, pop_size=50, generations=10, crossover_prob=0.8, mutation_prob=0.2):
        self.hypergraph = hypergraph
        self.pop_size = pop_size
        self.generations = generations
        self.crossover_prob = crossover_prob
        self.mutation_prob = mutation_prob
        self.num_objectives = 4
        # Generate reference points manually instead of using pymoo
        self.reference_points = self.generate_reference_points(num_points=10)

    def generate_reference_points(self, num_points=10):
        """Manually generate uniform reference points for 4 objectives"""
        if self.num_objectives != 4:
            raise ValueError("This implementation supports exactly 4 objectives")
            
        points = []
        # Create evenly distributed points in 4D space between [0,1]
        for i in range(num_points + 1):
            for j in range(num_points + 1 - i):
                for k in range(num_points + 1 - i - j):
                    l = num_points - i - j - k
                    if l < 0:
                        continue
                    point = np.array([i, j, k, l]) / num_points
                    points.append(point)
                    if len(points) >= num_points:
                        return np.array(points[:num_points])
        return np.array(points)

    def initialize_population(self, num_nodes, num_communities=5):
        """Initialize population"""
        population = []
        num_hyperedges = len(self.hypergraph[0])
        
        for _ in range(self.pop_size):
            struct_genes = np.random.randint(0, 2, size=num_hyperedges)
            comm_genes = np.random.randint(0, num_communities, size=num_nodes)
            population.append((struct_genes, comm_genes))
        
        return population

    def crossover_operation(self, parent1, parent2):
        """Crossover operation"""
        struct1, comm1 = parent1
        struct2, comm2 = parent2
        
        crossover_idx = np.random.randint(1, len(struct1))
        child_struct = np.concatenate([struct1[:crossover_idx], struct2[crossover_idx:]])
        
        crossover_mask = np.random.randint(0, 2, size=len(comm1))
        child_comm = np.where(crossover_mask == 1, comm1, comm2)
        
        return (child_struct, child_comm)

    def mutation_operation(self, individual, num_communities=5):
        """Mutation operation"""
        struct_genes, comm_genes = individual
        
        mutated_struct = np.where(
            np.random.rand(len(struct_genes)) < self.mutation_prob,
            1 - struct_genes,
            struct_genes
        )
        
        mutated_comm = comm_genes.copy()
        if np.random.rand() < self.mutation_prob:
            mutate_count = max(1, int(0.1 * len(comm_genes)))
            mutate_nodes = np.random.choice(len(comm_genes), size=mutate_count, replace=False)
            mutated_comm[mutate_nodes] = np.random.randint(0, num_communities, size=mutate_count)
        
        return (mutated_struct, mutated_comm)

    def non_dominated_sorting(self, population, objectives):
        """Non-dominated sorting"""
        num_individuals = len(population)
        domination_count = [0] * num_individuals
        dominated_by = [[] for _ in range(num_individuals)]
        fronts = []
        
        for i in range(num_individuals):
            for j in range(num_individuals):
                if i == j:
                    continue
                i_dominates_j = all(objectives[k][i] >= objectives[k][j] for k in range(self.num_objectives))
                j_dominates_i = all(objectives[k][j] >= objectives[k][i] for k in range(self.num_objectives))
                
                if i_dominates_j and not j_dominates_i:
                    dominated_by[i].append(j)
                    domination_count[j] += 1
        
        current_front = [i for i in range(num_individuals) if domination_count[i] == 0]
        
        while current_front:
            fronts.append(current_front)
            next_front = []
            for ind_idx in current_front:
                for dominated_idx in dominated_by[ind_idx]:
                    domination_count[dominated_idx] -= 1
                    if domination_count[dominated_idx] == 0:
                        next_front.append(dominated_idx)
            current_front = next_front
        
        return fronts

    def environmental_selection(self, population, objectives):
        """Environmental selection"""
        fronts = self.non_dominated_sorting(population, objectives)
        selected_indices = []
        
        for front in fronts:
            if len(selected_indices) + len(front) <= self.pop_size:
                selected_indices.extend(front)
            else:
                remaining_slots = self.pop_size - len(selected_indices)
                front_objectives = np.array([
                    [objectives[k][ind_idx] for k in range(self.num_objectives)] 
                    for ind_idx in front
                ])
                max_obj = np.max(front_objectives, axis=0, keepdims=True)
                normalized_front = front_objectives / max_obj
                normalized_ref = self.reference_points / max_obj
                distances = np.sqrt(np.sum(
                    (normalized_front[:, None, :] - normalized_ref[None, :, :]) ** 2, 
                    axis=2
                ))
                min_distances = np.min(distances, axis=1)
                top_individuals = np.argsort(min_distances)[:remaining_slots]
                selected_indices.extend([front[idx] for idx in top_individuals])
                break
        
        return [population[idx] for idx in selected_indices]

    def calculate_hypervolume(self, obj_values, ref_point):
        """Calculate hypervolume"""
        return np.prod([obj - ref for obj, ref in zip(obj_values, ref_point)])

    def optimize(self, num_nodes, num_communities=5):
        """Execute NSGA-III optimization"""
        population = self.initialize_population(num_nodes, num_communities)
        prev_community = {}
        role_hierarchy = {node: random.randint(1, 4) for node in self.hypergraph[0].keys()}
        
        for gen in range(self.generations):
            objectives = [[0.0 for _ in range(len(population))] for _ in range(self.num_objectives)]
            for ind_idx, individual in enumerate(population):
                struct_genes, comm_genes = individual
                activated_hyperedges = {}
                hyperedge_list = list(self.hypergraph[0].items())
                for he_idx, (he_id, he_data) in enumerate(hyperedge_list):
                    if struct_genes[he_idx] == 1:
                        activated_hyperedges[he_id] = he_data
                
                community_struct = {c: [] for c in range(num_communities)}
                node_list = list(self.hypergraph[0].keys())
                for node_idx, comm_id in enumerate(comm_genes):
                    if node_idx < len(node_list):
                        community_struct[comm_id].append(node_list[node_idx])
                community_struct = {c: nodes for c, nodes in community_struct.items() if len(nodes) > 0}
                
                if not community_struct:
                    qh, jc, sd, re = 0.0, 0.0, 0.0, 1e6
                else:
                    qh = compute_hypergraph_modularity(activated_hyperedges, community_struct)
                    jc = compute_interlayer_consistency(community_struct)
                    sd = compute_dynamic_stability(prev_community, community_struct)
                    re = compute_resource_efficiency(activated_hyperedges, role_hierarchy)
                
                objectives[0][ind_idx] = qh
                objectives[1][ind_idx] = jc
                objectives[2][ind_idx] = sd
                objectives[3][ind_idx] = -re
            
            selected_parents = self.environmental_selection(population, objectives)
            offspring = []
            while len(offspring) < self.pop_size - len(selected_parents):
                parent1, parent2 = random.sample(selected_parents, 2)
                if random.random() < self.crossover_prob:
                    child = self.crossover_operation(parent1, parent2)
                else:
                    child = parent1
                child = self.mutation_operation(child, num_communities)
                offspring.append(child)
            
            population = selected_parents + offspring
            prev_community = community_struct.copy()
            
            print(f"Generation {gen+1}/{self.generations} | Best Qh: {max(objectives[0]):.4f}, Best Jc: {max(objectives[1]):.4f}")
        
        final_objectives = []
        final_communities = []
        for individual in population:
            struct_genes, comm_genes = individual
            community_struct = {c: [] for c in range(num_communities)}
            node_list = list(self.hypergraph[0].keys())
            for node_idx, comm_id in enumerate(comm_genes):
                if node_idx < len(node_list):
                    community_struct[comm_id].append(node_list[node_idx])
            community_struct = {c: nodes for c, nodes in community_struct.items() if len(nodes) > 0}
            final_communities.append(community_struct)
            
            qh = compute_hypergraph_modularity(self.hypergraph[0], community_struct)
            jc = compute_interlayer_consistency(community_struct)
            sd = compute_dynamic_stability(prev_community, community_struct)
            re = compute_resource_efficiency(self.hypergraph[0], role_hierarchy)
            final_objectives.append([qh, jc, sd, re])
        
        final_objs_np = np.array(final_objectives)
        ref_point = np.min(final_objs_np, axis=0)
        final_objs_for_hv = final_objs_np.copy()
        final_objs_for_hv[:, 3] = ref_point[3] - final_objs_for_hv[:, 3]
        hv_values = [self.calculate_hypervolume(obj, ref_point) for obj in final_objs_for_hv]
        best_idx = np.argmax(hv_values)
        best_individual = population[best_idx]
        best_community = final_communities[best_idx]
        
        print("\nOptimization Complete! Optimal Individual Objectives:")
        print(f"Hypergraph Modularity (Qh): {final_objectives[best_idx][0]:.4f}")
        print(f"Inter-layer Consistency (Jc): {final_objectives[best_idx][1]:.4f}")
        print(f"Dynamic Stability (Sd): {final_objectives[best_idx][2]:.4f}")
        print(f"Resource Efficiency (Re): {final_objectives[best_idx][3]:.4f}")
        print(f"Optimal Community Structure: {best_community}")
        
        return best_individual, best_community


# -------------------------- Test Code (Main Function) -------------------------
if __name__ == "__main__":
    # Set random seeds
    random.seed(42)
    np.random.seed(42)
    
    # Generate synthetic data
    num_nodes = 30
    nodes = [f"node_{i}" for i in range(num_nodes)]
    layers = ["R&D", "Marketing", "Management"]
    initial_hyperedges = {}
    for i in range(20):
        he_id = f"he_initial_{i}"
        selected_nodes = random.sample(nodes, k=4)
        layer = layers[i % 3]
        initial_hyperedges[he_id] = (
            {'nodes': selected_nodes, 'type': 'strategic'},
            layer,
            0,
            1.0
        )
    time_windows = [0, 1, 2, 3]
    
    # Construct dynamic hypergraph
    hypergraph_builder = DynamicHypergraph(nodes, layers, initial_hyperedges, time_windows)
    hypergraph_data = hypergraph_builder.construct_hypergraph(current_time=1)
    print(f"Hypergraph Constructed at Time=1 | Hyperedges: {len(hypergraph_data[0])} | Layers: {len(hypergraph_data[1])}")
    
    # Run NSGA-III community detection
    nsga3_detector = NSGAIIICommunityDetector(
        hypergraph=hypergraph_data,
        pop_size=50,
        generations=10,
        crossover_prob=0.8,
        mutation_prob=0.2
    )
    optimal_individual, optimal_community = nsga3_detector.optimize(num_nodes=num_nodes, num_communities=5)
    
    # Print results
    print("\nOptimal Individual Genes:")
    print(f"Structure Genes: {optimal_individual[0]}")
    print(f"Community Genes: {optimal_individual[1]}")


Hypergraph Constructed at Time=1 | Hyperedges: 22 | Layers: 3
Generation 1/10 | Best Qh: -0.8000, Best Jc: 0.5000
Generation 2/10 | Best Qh: -0.8000, Best Jc: 0.5000
Generation 3/10 | Best Qh: -0.8000, Best Jc: 0.5000
Generation 4/10 | Best Qh: -0.8000, Best Jc: 0.5000
Generation 5/10 | Best Qh: -0.8000, Best Jc: 0.5000
Generation 6/10 | Best Qh: -0.8000, Best Jc: 0.5000
Generation 7/10 | Best Qh: -0.8000, Best Jc: 0.5000
Generation 8/10 | Best Qh: -0.8000, Best Jc: 0.5000
Generation 9/10 | Best Qh: -0.8000, Best Jc: 0.5000
Generation 10/10 | Best Qh: -0.8000, Best Jc: 0.5000

Optimization Complete! Optimal Individual Objectives:
Hypergraph Modularity (Qh): -0.8000
Inter-layer Consistency (Jc): 0.5000
Dynamic Stability (Sd): 0.1423
Resource Efficiency (Re): 9.6071
Optimal Community Structure: {0: ['he_initial_0', 'he_initial_3', 'he_initial_4', 'he_initial_5', 'he_initial_17'], 1: ['he_initial_14', 'he_initial_18', 'he_initial_19', 'he_new_1_0'], 2: ['he_initial_2', 'he_initial_12', 'h